In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import missingno as msno # display missing data

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # Alignments
%matplotlib inline

import seaborn as sns # theme & dataset
print(f"Matplotlib Version : {mpl.__version__}")
print(f"Seaborn Version : {sns.__version__}")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Coloumn Description

* **Suburb** - Suburb

* **Address** - Address

* **Rooms** - Number of rooms

* **Price** - Price in Australian dollars

**Method** - 
    
    S - property sold;
    SP - property sold prior;
    PI - property passed in;
    PN - sold prior not disclosed;
    SN - sold not disclosed;
    NB - no bid;
    VB - vendor bid;
    W - withdrawn prior to auction;
    SA - sold after auction;
    SS - sold after auction price not disclosed.
    N/A - price or highest bid not available.

**Type** -

    br - bedroom(s);
    h - house,cottage,villa, semi,terrace;
    u - unit, duplex;
    t - townhouse;
    dev site - development site;
    o res - other residential.

* **SellerG** - Real Estate Agent

* **Date** - Date sold

* **Distance** - Distance from CBD in Kilometres

* **Regionname** - General Region (West, North West, North, North east …etc)

* **Propertycount** - Number of properties that exist in the suburb.

* **Bedroom2** - Scraped # of Bedrooms (from different source)

* **Bathroom** - Number of Bathrooms

* **Car** - Number of carspots

* **Landsize** - Land Size in Metres

* **BuildingArea** - Building Size in Metres

* **YearBuilt** - Year the house was built

* **CouncilArea** - Governing council for the area

* **Lattitude** - Self explanitory

* **Longtitude** - Self explanitory

The objects of this notebook:

1. Defien Queistions
2. Understand Data
3. Pre-processing
4. Visualisation
5. Clustering
6. Linear Regression

# 1. Define Quetions

Clustering

Linear Regression

# 2. Understand Data

* **Number of rows/entries** - 34857 entries, 0 to 34856

* **Data columns** (total 21 columns):

*   **Numerical** (13 columns)
 *    float64 (12 columns) - 
    'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
    'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
    'Propertycount'

 *    int64 (1 column) - 'Rooms'

* **Nominal/Categorical** (8 columns)
 *     object (8 columns) - 'Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea', 'Regionname'
 

### Which features could higly be relative to Price?

Inference - Distance, Landsize, BuildingArea, YearBuilt, Rooms


In [ ]:
# show the current working directory
%pwd 

In [34]:
df = pd.read_csv('/kaggle/input/melbourne-housing-market/Melbourne_housing_FULL.csv')
df.shape # 34857 rows, 21 columns

In [ ]:
df.head()

In [ ]:
# df.dtypes
df.info()

In [ ]:
# Statistical Summary
df.describe()

In [ ]:
# Summary of object features
# df.describe(include = 'object')
df.describe(include = [np.object])

In [ ]:
len(df['Suburb'].unique())

In [ ]:
# SELECT h FROM Type
df_type_h = df.loc[df['Type'] == 'h']
df_type_h.shape # 23980, 21

In [ ]:
# the number of values that each unique value has
df['Type'].value_counts()
df['Price'].value_counts().head(10)

There are not many empty values in **categorical features**, but there are a lot of empty values in **numerical features**.

In [ ]:
print(df.isnull().sum())
print('')
print(df.isnull().sum()/len(df))

In [ ]:
msno.matrix(df)

# EDA

* Missing values
* Outliers


In [ ]:
[feature for feature in df if df[feature].dtype == 'O'] # O means object? list
[feature for feature in df if df[feature].dtype != 'O'] # numerical features

df.select_dtypes(include = 'object') # dataframe
df.select_dtypes(include = 'object').columns.to_list()

df.select_dtypes(exclude = 'object')

### Separate the dataset into categorical and numerical.

In [63]:
cat_cols = [col for col in df if df[col].dtype == 'O']

print('Number of categorical values: ',len(cat_cols))

df[cat_cols].head()

In [6]:
for col in df[cat_cols]:
    print(f'{col} has {len(df[col].unique())} distict values.')

In [7]:
cp_cat_cols = ['Type', 'Method', 'CouncilArea', 'Regionname']

In [122]:
for col in df[cp_cat_cols]:
    data = df.copy()
    data.groupby(col)['Price'].median().plot.bar()
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.title(col)
    plt.show()

In [3]:
num_cols = [col for col in df if df[col].dtype != 'O']

print('Number of numerical values: ',len(num_cols))

df[num_cols].head()

#### Divide numerical values into two groups: discrete and continuous

In [8]:
disc_cols = [col for col in num_cols if len(df[col].unique())<25]
df[disc_cols].head()

In [9]:
cont_cols = [col for col in num_cols if col not in disc_cols]
df[cont_cols].head()

In [10]:
for col in df[cont_cols]:
    print(col, len(df[col].unique())) 

In [127]:
# discrete features and Price

for col in disc_cols:
    data = df.copy()
    data.groupby(col)['Price'].median().plot.bar()
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.title(col)
    plt.show()

In [129]:
# Continuous features - with histogram

for col in cont_cols:
    data = df[col]
    df[col].hist(bins = 25)
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.title(col)
    plt.show()

In [130]:
plt.scatter(df['BuildingArea'], df['Price'])
plt.xlim(0, 1000)

In [131]:
# Using logarithmic transformation
for col in cont_cols:
    data = df.copy()
    if 0 in data[col].unique():
        pass
    else:
        data[col] = np.log(abs(data[col])) # To change Lattitude to positive, abs is used
        data['Price'] = np.log(data['Price'])
        
        plt.scatter(data[col], data['Price'])
        plt.xlabel(col)
        plt.ylabel('Price')
        plt.title(col)
        plt.show()

#### Outliers

In [61]:
for col in cont_cols:
    data = df.copy()
    if 0 in data[col].unique():
        pass
    else:
        data[col] = np.log(abs(df[col]))
        data.boxplot(column=col)
        plt.ylabel(col)
        plt.show()

# 3. Feature Engineering

* Drop unnecessary columns
* Remove/Fill missing values
* Imputation
* One hot encoding ?



Unncessary features? - Car, Address, SellerG, Postcode, Lattitude, Longitude, Propertycount

In [35]:
print(df.isnull().sum())
print('')
print(df.isnull().sum()/len(df))

In [88]:
df[df['Distance'].isnull()]

In [37]:
cols_with_nan = [col for col in df.columns if df[col].isnull().sum() > 1 and df[col].dtypes == "O"]

for col in cols_with_nan:
    print("{}: {}% missing values." .format(col, np.round(df[col].isnull().mean(),4)))

In [39]:
for col in cols_with_nan:
    df[col].fillna(df[col].mode()[0], inplace = True)

df[cols_with_nan].isnull().sum()

In [40]:
# Fill Distance, Postcode, and Propertycount with mode as there are only 1 and 3 missing values
df['Propertycount'].fillna(df['Propertycount'].mode()[0], inplace = True)
df['Distance'].fillna(df['Distance'].mode()[0], inplace = True)
df['Postcode'].fillna(df['Postcode'].mode()[0], inplace = True)

In [41]:
# Find numerical features havig nan 
numerical_with_nan = [col for col in df.columns if df[col].isnull().sum()>1 and df[col].dtypes != 'O']

# mean calculates (number of True values) / number of entire values 
for col in numerical_with_nan:
    print("{}: {}% missing value" .format(col, np.around(df[col].isnull().mean(),4)))

In [42]:
df['Price'].head()

In [51]:
df.loc[df['Price'] == np.isnan(df['Price'])]

In [54]:
# df[df['Price'].isnull()]
df[np.isnan(df['Price'])]

In [56]:
# Replace the missing values

for col in numerical_with_nan:
    # Replace with median as there are many outliers
    median_value = df[col].median()

    # Create a new feature to capture nan values
    df[col+'nan'] = np.where(df[col].isnull(),1,0)
    df[col].fillna(median_value, inplace = True)
    
df[numerical_with_nan].isnull().sum()

In [58]:
df.head()

In [59]:
df.loc[df['Pricenan'] == 1]

# Handling Rare Categorical Feature
Remove categorical variables that are present less than 1% of the observations

In [66]:
# Remove categorical variables that are present less than 1% of the observations
for col in cat_cols:
    temp = df.groupby(col)['Price'].count()/len(df)
    temp_df = temp[temp>0.01].index
#     print(temp)
    df[col] = np.where(df[col].isin(temp_df), df[col], 'Rare_var')

df.head()

In [67]:
df['Date'].value_counts()

In [ ]:
# Address, SellerG, Postcode, Lattitude, Longtitud
# df.dropna(axis = 0, subset = )
df = df.drop(columns = ['Address', 'Car', 'SellerG', 'Postcode', 'Lattitude', 'Longtitude', 'Propertycount'], axis = 1)
df.head()

In [ ]:
df = df.dropna(axis = 0, subset = ['Price', 'BuildingArea', 'YearBuilt'])
df.isnull().sum()

In [ ]:
print(len(df))
df.corr()
sns.heatmap(df.corr())

In [ ]:
df['Landsize'].fillna(round(df['Landsize'].median()), inplace = True)
# print(len(df.loc[df['Landsize'] == df['Landsize'].median()]))
# len(df.loc[df['Landsize'] == round(df['Landsize'].mean())])
print(len(df)) # 10249
df.isnull().sum()

In [ ]:
# pd.set_option('display.max_rows', 50)

# Visualisation

Suburb, YearBuilt, CouncilArea, Regionname

In [ ]:
for i in df:
    plt.hist(df[i])
    plt.title(i)
    plt.show()

Suburb, YearBuilt, CouncilArea, and Regionname are not shown clearly

In [ ]:
# Suburb, YearBuilt, CouncilArea, and Regionname
df['cat_suburb'] = df['Suburb'].astype('category').cat.codes
df['cat_yeaerbuilt'] = df['YearBuilt'].astype('category').cat.codes
df['cat_councilarea'] = df['CouncilArea'].astype('category').cat.codes
df['cat_regionname'] = df['Regionname'].astype('category').cat.codes
df.head()

In [ ]:
cat_df = df[['Rooms', 'Type', 'Price', 'Method', 'Date', 'Distance',
       'Bedroom2', 'Bathroom', 'Landsize', 'BuildingArea', 'cat_suburb', 'cat_yeaerbuilt',
       'cat_councilarea', 'cat_regionname']]
cat_df.head()

In [ ]:
for i in cat_df:
    plt.hist(cat_df[i])
    plt.title(i)
    plt.show()

In [ ]:
plt.figure(figsize = (12, 5)) # width, height
plt.subplot(121)
plt.bar(df['BuildingArea'], df['Price'])
# plt.gcf().axes[0].yaxis.get_major_formatter().set_scientific(False)
plt.ticklabel_format(style = "plain")
plt.subplot(122)
plt.scatter(df['BuildingArea'], df['Price'])
plt.suptitle('BuildingArea and Price')
plt.ticklabel_format(style = "plain")
plt.show()

# df_converted = df

# for col_name in df_converted:
#     if(df_converted[col_name].dtype == 'object'):
#         df_converted[col_name] = df_converted[col_name].astype('category')
#         df_converted[col_name] = df_converted[col_name].cat.codes

In [ ]:
for i in cat_df:
    sns.barplot(cat_df[i].value_counts().index, cat_df[i].value_counts()).set_title(i)
    plt.show()

Set price range? for visualisation?

In [ ]:
df_price = pd.DataFrame(df['Price'])

In [ ]:
# df_price.apply(lambda x : for i in x if i < 5000)

def combine_price(x):
    if x <= 500000:
        return 500000
    elif x > 500000 and x <= 1000000:
        return 1000000
    elif x > 1000000 and x <= 1500000:
        return 1500000
    elif x > 1000000 and x <= 2000000:
        return 2000000
    elif x > 1000000 and x <= 2500000:
        return 2500000
    else:
        return 3000000

# Create df_price
df_price = pd.DataFrame(df['Price'])
    
# Create price range
df_price['price_range'] = df_price["Price"].apply(combine_price)
df_price['price_range'].value_counts()
df_price.head()

In [ ]:
# Add price_range to df
df['price_range'] = df_price['price_range']
df.head()

In [ ]:
# Take unique values (price_range) and the number of value of each unique value (counts) in df_price
price_data = {'price_range': df_price['price_range'].unique(), 'counts': df_price['price_range'].value_counts()}
# Create a DataFrame called df_rpice_range with price_data 
df_price_range = pd.DataFrame(data = price_data)
df_price_range

In [ ]:
# Sort price_range and rename index
df_price_range.sort_values(by = 'price_range', inplace = True)
df_price_range.rename(index = {2000000:0, 500000:1, 1000000:2, 1500000:3, 2500000:4, 3000000:5}, inplace = True)
df_price_range

I just wanted to plot a Pareto diagram... 

In [ ]:
# https://stackoverflow.com/questions/53577630/how-to-make-pareto-chart-in-python
df_price_range["cumpercentage"] = df_price_range["counts"].apply(lambda x: x/len(df["Price"])*100).cumsum()

fig, ax = plt.subplots()
ax.bar(df_price_range['price_range'].index, df_price_range["counts"], color = "C0")
x = np.arange(len(df_price_range))
xl = df_price_range['price_range']
ax.set_xticks(x)
ax.set_xticklabels(xl)
# ax.ticklabel_format(style = "plain")

ax2 = ax.twinx()
ax2.plot(df_price_range['price_range'].index, df_price_range["cumpercentage"], color = "C1", marker = "D", ms = 7)

ax.tick_params(axis = "y", colors = "C0")
ax2.tick_params(axis = "y", colors = "C1")
plt.show()

df_price_range

In [ ]:
df['BuildingArea'].value_counts()

In [ ]:
# Compare price_range and other categorical features
print(pd.pivot_table(df, index = 'price_range', columns = 'Type', values = 'cat_regionname', aggfunc = 'count'))
print()
# print(pd.pivot_table(df, index = 'price_range', columns = 'BuildingArea'))

In [ ]:
# normal distribution?
# positive/right skwness - outliers lying on the right side

# sns.distplot(df['Price']) # distplot - displot or histplot
sns.histplot(df['Price'])
plt.ticklabel_format(style='plain', useOffset=False)
# plt.ticklabel_format(style='plain', axis='x',useOffset=False)

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
type(iris) # sklearn.utils.Bunch
len(iris.data) # 150
iris.data #nump.ndarray

In [ ]:
from sklearn.cluster import KMeans
X = df[['Price', 'BuildingArea']]

kmeans = KMeans(n_clusters = 6, random_state = 0).fit(X)
labels = kmeans.labels_
# type(kmeans.labels_) #np.ndarray

In [ ]:
a = pd.Series(data = labels)
a.value_counts()

In [ ]:
df_price['BuildArea'] = df['BuildingArea']
df_price['labels'] = pd.Series(data = labels)
df_price.head()

In [ ]:
df_price['labels'].value_counts()

In [ ]:
len(df_price['labels'].isna())

In [ ]:
# ax, fig = plt.figure(2, figsize = (5, 3))
plt.scatter(X['BuildingArea'], X['Price'], c=labels.astype(float), edgecolor = 'k')
plt.ticklabel_format(style = 'plain')
plt.show()